In [1]:
import os
import ee
ee.Initialize()
import pandas as pd
import geopandas as gpd
import geemap.eefolium as emap

import folium
from osgeo import ogr, osr

In [2]:
os.chdir("C:/Users/liang/Desktop/...")

pre17 = gpd.read_file("xxx.shp")
print("shape of presence 2017 is", pre17.shape)
pre17.head

shape of presence 2017 is (10, 8)


<bound method NDFrame.head of    Cell_numbe           STR        Trap  PA_Plants_   Latitude  Longitude  \
0       899.0  Fort Niagara   7519-10.2   1033795.0  43.257000 -79.052900   
1       895.0  Fort Niagara   7519-10.1   1033794.0  43.257700 -79.053500   
2       886.0     River Rd.   7385-10.8   1033791.0  43.232000 -79.050900   
3       660.0     Joseph D.   7251-10.6   1033666.0  43.218500 -79.039900   
4       647.0     Joseph D.   7251-10.2   1033660.0  43.218500 -79.045500   
5       645.0     Joseph D.   7251-10.1   1033659.0  43.218600 -79.046298   
6       883.0      River Rd   7385-10.7   1033790.0  43.235700 -79.051900   
7       673.0     Joseph D.   7251-10.1   1033670.0  43.218700 -79.037400   
8       654.0     Joseph D.   7251-10.4   1033663.0  43.218500 -79.042100   
9         0.0   Mississauga  Approx Loc         0.0  43.566905 -79.673292   

     Year                         geometry  
0  2017.0  POINT (1023204.645 1187093.361)  
1  2017.0  POINT (1023046.247 11

In [3]:
pre17b = pre17.drop([9, ],axis=0)
pre18 = gpd.read_file("xxx2.shp")
pre19 = gpd.read_file("xxx3.shp")

print("shape of presence 2018 is", pre18.shape)
print("shape of presence 2019 is", pre19.shape)

shape of presence 2018 is (614, 34)
shape of presence 2019 is (1115, 41)


In [4]:
def getCrds_Y(gdf_pts):
    crds_Y = gdf_pts[['Latitude', 'Longitude','Year']]
    return (crds_Y)

crds = getCrds_Y(pre17b).append(getCrds_Y(pre18)).append(getCrds_Y(pre19))
crds17 = getCrds_Y(pre17b)
crds18 = getCrds_Y(pre18)
crds19 = getCrds_Y(pre19)

print(crds.shape)
print(crds.head)

ctd = crds[['Latitude', 'Longitude']].mean()
ctd

(1738, 3)
<bound method NDFrame.head of        Latitude  Longitude    Year
0     43.257000 -79.052900  2017.0
1     43.257700 -79.053500  2017.0
2     43.232000 -79.050900  2017.0
3     43.218500 -79.039900  2017.0
4     43.218500 -79.045500  2017.0
...         ...        ...     ...
1110  43.369675 -78.479308  2019.0
1111  43.368460 -78.488302  2019.0
1112  43.367376 -78.473551  2019.0
1113  43.367043 -78.235905  2019.0
1114  43.114663 -78.921735  2019.0

[1738 rows x 3 columns]>


Latitude     43.257856
Longitude   -78.844063
dtype: float64

In [5]:
map1 = folium.Map(location=[ctd['Latitude'],ctd['Longitude']], zoom_start=10, tiles='OpenStreetMap')

def addmap(crds, map1,color):
    for i in range(0, len(crds)):
        folium.Marker([crds.iloc[i]['Latitude'], crds.iloc[i]['Longitude']] ,icon=folium.Icon(color=color)).add_to(map1)
addmap(crds19, map1, "purple")
addmap(crds18, map1, "blue")
addmap(crds17, map1, "red")

map1

In [6]:
eemap = emap.Map(center= [ctd['Latitude'], ctd['Longitude']], zoom=10)
eemap.add_basemap('SATELLITE')

def addmap2(crds, eemap, color):
    crds2 = crds[['Longitude','Latitude']]
    pts = ee.List(crds2.values.tolist())
    pts2 = ee.Geometry.MultiPoint(pts)
    eemap.addLayer(pts2,{'color':color})

addmap2(crds19, eemap,"purple")
addmap2(crds18, eemap,"blue")
addmap2(crds17, eemap,"red")

eemap

In [7]:
crds.describe()

,Latitude,Longitude,Year
count,1738.000000,1738.000000,1738.000000
mean,43.257856,-78.844063,2018.636364
std,0.078948,0.203949,0.491834
min,43.006536,-79.067857,2017.000000
25%,43.208790,-79.032486,2018.000000
50%,43.271726,-78.858610,2019.000000
75%,43.323403,-78.695117,2019.000000
max,43.374536,-78.181164,2019.000000


In [8]:
aoi = ee.Geometry.Polygon([
    [-79.068, 43.375],
    [-78.181, 43.375],
    [-78.181, 43.005],
    [-79.068, 43.005]
])

aoi = ee.Geometry.Polygon([
    [-76.068, 43.375],
    [-75.181, 43.375],
    [-75.181, 42.005],
    [-76.068, 42.005]
])

#aoi = ee.Projection('EPSG:4326')
centroid = aoi.centroid()
long, lat = centroid.getInfo()['coordinates']
print ('Centroid: long = {}, lat = {}'.format(long, lat))

Centroid: long = -75.62449999999973, lat = 42.68833703924229


In [9]:
cpl = ee.ImageCollection('USDA/NASS/CDL')
#cpl = ee.ImageCollection('USDA/NAIP/DOQQ')
cpl2 = cpl.filterBounds(aoi).filterDate('2015-01-01', '2020-05-05')

def subset(image):
    return image.select(['cropland'])

crls = cpl2.map(subset)
print("Band name is: " + str(crls.first().bandNames().getInfo()))
crls.size().getInfo()

Band name is: ['cropland']


5

In [10]:
ls = crls.toList(crls.size().getInfo())
crop1 = ee.Image(ls.get(3)).select(['cropland'])
ee.Image(ls.get(3)).bandNames().getInfo()

['cropland']

In [11]:
map = emap.Map(center=[lat, long],zoom=6)
map.add_basemap('SATELLITE')
#map.addLayer(cpl3.randomVisualizer())
map.addLayer(crop1)
map

In [12]:
crls2 = crls.mosaic().clip(aoi)

crls_mode = crls.reduce(ee.Reducer.mode())
crls_aoi = crls_mode.clip(aoi)
crls_aoi.bandNames().getInfo()

['cropland_mode']

In [13]:
cpl_chry = crls_mode.updateMask(crls_mode.eq(69)) # cherry: 66, grape: 69

In [15]:
map2 = emap.Map(center=[lat, long],zoom=12)
map2.add_basemap('SATELLITE')
#map2.addLayer(crls_mode)
map2.addLayer(cpl_chry)
map2

In [24]:
task = ee.batch.Export.image.toDrive(image= crls_mode.toInt(), maxPixels = 1e13, description='CropLayer_Mode15_20',scale=100,folder='myFolder',fileFormat='GeoTIFF')
task.start()
task.status()['state']

'READY'

In [ ]:
import time
starttime=time.time()
state = "Ready"

n=0
while state!='COMPLETED' and state!= 'FAILED':
    print("status: " + task.status()['state'])
    state = task.status()['state']
    n += 1
    print("Time consumed: " + str(5*n) +'s')
    time.sleep(5.0 - ((time.time() - starttime) % 5.0))

if task.status()['state'] == 'FAILED':
    print (task.status()['error_message'])

status: READY
Time consumed: 5s
status: READY
Time consumed: 10s
status: READY
Time consumed: 15s
status: READY
Time consumed: 20s
status: READY
Time consumed: 25s
status: READY
Time consumed: 30s
status: READY
Time consumed: 35s
status: READY
Time consumed: 40s
status: RUNNING
Time consumed: 45s
status: RUNNING
Time consumed: 50s
status: RUNNING
Time consumed: 55s
status: RUNNING
Time consumed: 60s
status: RUNNING
Time consumed: 65s
status: RUNNING
Time consumed: 70s
status: RUNNING
Time consumed: 75s
status: RUNNING
Time consumed: 80s
status: RUNNING
Time consumed: 85s
status: RUNNING
Time consumed: 90s
status: RUNNING
Time consumed: 95s
status: RUNNING
Time consumed: 100s
status: RUNNING
Time consumed: 105s
status: RUNNING
Time consumed: 110s
status: RUNNING
Time consumed: 115s
status: RUNNING
Time consumed: 120s
status: RUNNING
Time consumed: 125s
status: RUNNING
Time consumed: 130s
status: RUNNING
Time consumed: 135s
status: RUNNING
Time consumed: 140s
status: RUNNING
Time consume

In [148]:
task.status()
cpl3.projection().nominalScale().getInfo()

111319.49079327357